In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Intro

There are a number of python packages to work with FCA. In this notebook we will compare their performances in the basic FCA task: constructing the concept lattice from a formal context.

We consider two packages: FCApy and Concepts

// More packages can be compared in the future

# Install competitors libraries

`FCApy` package (by Egor Dudyrev, HSE Moscow): https://github.com/EgorDudyrev/FCApy 

In [2]:
!pip -q install -U fcapy[context,lattice] --user

In [3]:
from fcapy import LIB_INSTALLED
from fcapy.context import FormalContext
from fcapy.lattice import ConceptLattice
from fcapy.visualizer import ConceptLatticeVisualizer

`Concepts` package (by Sebastian Bank, University of Leipzig): https://github.com/xflr6/concepts

In [4]:
!pip -q install -U concepts --user

In [5]:
import concepts

`fcapsy` package (by Tomáš Mikula, Palacký University): https://github.com/mikulatomas/fcapsy

Upd. We drop `fcapsy` package from the benchmark since now it uses `concepts` package under the hood

# Load data

First we load some classic FCA contexts (datasets)

In [6]:
frames_classic = {}
contexts_to_test = ['animal_movement', 'digits', 'gewaesser',
                    'lattice', 'liveinwater', 'tealady']

!rm -rf tmp
!mkdir tmp

for K_name in contexts_to_test:
    fname = f'tmp/{K_name}.cxt'
    !wget -O {fname} -q https://raw.githubusercontent.com/EgorDudyrev/FCApy/main/data/{K_name}.cxt
    df = FormalContext.read_cxt(fname).to_pandas()
    df.name = K_name
    frames_classic[K_name] = df
!rm -rf tmp

Add Bob-Ross dataset which has more objects and attributes than the classic FCA datasets

In [7]:
import pandas as pd
K_name = 'bob_ross'
fname = f"{K_name}.csv"
!wget -O {fname} -q https://raw.githubusercontent.com/fivethirtyeight/data/master/bob-ross/elements-by-episode.csv 
df = pd.read_csv(fname)
df['EPISODE_TITLE'] = df['EPISODE']+' '+df['TITLE']
df = df.drop(['EPISODE','TITLE'],1).set_index('EPISODE_TITLE').astype(bool)
df.name = K_name
frames_classic[K_name] = df
print(df.shape)
!rm {fname}

(403, 67)


These classic real world contexts are small so we add some big random contexts to our examination

In [8]:
import numpy as np
from itertools import product

np.random.seed(42)
n_objects_vars = [10, 30, 100]
n_attributes_vars = [10, 30, 50]
densities_vars = [0.1, 0.5, 0.9]
frames_random = {}
for comb in product(n_objects_vars, n_attributes_vars, densities_vars):
    n_objects, n_attributes, density = comb

    frame = pd.DataFrame(np.random.binomial(1, density, size=(n_objects,n_attributes)))
    frame.columns = [f"m_{i}" for i in frame.columns]
    frame.index = [f"g_{i}" for i in frame.index]
    frame = frame.astype(bool)

    frame.name = f"random_{n_objects}_{n_attributes}_{density}"
    frames_random[frame.name] = frame

In [9]:
frames = dict(frames_classic, **frames_random)
#frames = dict(frames_classic)

# Run benchmarks

## Default lattice visualizations

Let us take one classic FCA context 'animal movement' and a bigger one 'bob ross' dataset

The description of Animals context:
* objects (rows) are Animals
* attributes (columns) are Actions
* the table shows whether an Animal can perform an Action

The description of Bob Ross dataset:
* objects (rows) are paintings by Bob Ross
* attributes (columns) are specific elements in these paintings
* the table shows whether an element is on a painting

In [10]:
K_names = ['animal_movement', 'tealady']#'bob_ross']

In [11]:
K_name = K_names[0]
print(K_name)
df = frames[K_name]
print(df.shape)
df.head()

animal_movement
(16, 4)


,fly,hunt,run,swim
dove,True,False,False,False
hen,False,False,False,False
duck,True,False,False,True
goose,True,False,False,True
owl,True,True,False,False


### Visualization by `concepts`

The visualization can be found in the file
* _lattice_visualization_concepts_animal_movement.png_
* _lattice_visualization_concepts_bob_ross.png_

In [12]:
from datetime import datetime

In [13]:
for K_name in K_names:
    df = frames[K_name]
    print(K_name)
    t1 = datetime.now()
    K_concepts = concepts.Context(df.index, df.columns, df.values)
    L_concepts = K_concepts.lattice
    print(f'Lattice constructed in {(datetime.now()-t1).total_seconds()} seconds')
    L_concepts.graphviz(f'imgs/lattice_visualization/concepts_{K_name}', render=True);
    t2 = datetime.now()
    dt = (t2-t1).total_seconds()
    print(f"Executed in {dt} seconds")

animal_movement
Lattice constructed in 0.001552 seconds
Executed in 0.041914 seconds
tealady
Lattice constructed in 0.009563 seconds
Executed in 0.058486 seconds


### Visualization by `fcapy`

Visualizations can be found in the files
* _lattice_visualization_fcapy_networkx_animal_movement.png_ 
* _lattice_visualization_fcapy_plotly_animal_movement.png_

* _lattice_visualization_fcapy_networkx_bob_ross.png_ 
* _lattice_visualization_fcapy_plotly_bob_ross.png_

In [14]:
import matplotlib.pyplot as plt

for K_name in K_names:
    df = frames[K_name]
    print(K_name)
    
    t1 = datetime.now()
    K_fcapy = FormalContext.from_pandas(df)
    L_fcapy = ConceptLattice.from_context(K_fcapy)
    print(f'Lattice constructed in {(datetime.now()-t1).total_seconds()} seconds')
    vsl_fcapy = ConceptLatticeVisualizer(L_fcapy)
    print(f'Visualizer constructed in {(datetime.now()-t1).total_seconds()} seconds')

    plt.title('Networkx lattice')
    vsl_fcapy.draw_networkx()
    plt.savefig(f'imgs/lattice_visualization/fcapy_networkx_{K_name}.png')
    plt.close()
    print(f'Png saved in {(datetime.now()-t1).total_seconds()} seconds')

animal_movement
Lattice constructed in 0.001433 seconds
Visualizer constructed in 0.001603 seconds
Png saved in 0.2417 seconds
tealady
Lattice constructed in 0.030366 seconds
Visualizer constructed in 0.03093 seconds
Png saved in 0.856946 seconds


## Time to construct a lattice

Functions to run the same lattice construction task with different libraries

In [15]:
from datetime import datetime

In [16]:
from fcapsy import basic_level

In [17]:
def construct_context_by_lib(frame, lib_name):
    if lib_name == 'concepts':
        K = concepts.Context(frame.index, frame.columns, frame.values)
    elif lib_name == 'fcapy':
        K = FormalContext.from_pandas(frame)
    #elif lib_name == 'fcapsy':
    #    K = fcapsy.Context.from_pandas(frame)
    else:
        raise ValueError(f'Given library "{lib_name}" is not supported')
        
    return K

In [18]:
def test_intent_extent_time_by_func(objects, attributes, extent_func, intent_func, samples_per_size=100):
    times = []
    for arr, fnc in [(objects, intent_func), (attributes, extent_func)]:
        subsample_sizes = np.logspace(0, np.log(len(arr))/np.log(10), 10)
        subsample_sizes = subsample_sizes.round(0).astype(int)    
        np.random.seed(42)
        samples = [
            sample
            for size in subsample_sizes
            for sample in np.random.choice(arr, size=(samples_per_size, size))
        ]
        
        t1 = datetime.now()
        intents = [fnc(sample) for sample in samples]
        t2 = datetime.now()
        dt = (t2-t1).total_seconds()/len(samples) 
        times.append(dt)
    intent_time, extent_time = times
    
    return intent_time, extent_time


def test_intent_extent_time_by_lib(frame, K, lib_name, samples_per_size=100):
    if lib_name == 'concepts':
        intent_time, extent_time = test_intent_extent_time_by_func(
            frame.index, frame.columns, K.extension, K.intension, samples_per_size)
    elif lib_name == 'fcapy':
        intent_time, extent_time = test_intent_extent_time_by_func(
            frame.index, frame.columns, K.extension, K.intention, samples_per_size)
    #elif lib_name == 'fcapsy':
    #    intent_time, extent_time = test_intent_extent_time_by_func(
    #        frame.index, frame.columns,
    #        lambda ar: K.down(K.Attributes(ar)),
    #        lambda ar: K.up(K.Objects(ar)),
    #        samples_per_size
    #    )
    else:
        raise ValueError(f'Given library "{lib_name}" is not supported')
        
    return intent_time, extent_time


def test_intent_extent_time_by_lib_multiprocess(
    frame, K, lib_name, intent_time, extent_time, samples_per_size=100
):
    intent_time.value, extent_time.value = test_intent_extent_time_by_lib(
        frame, K, lib_name, samples_per_size
    )

In [19]:
def test_lattice_time_by_func(K, L_func):
    t1 = datetime.now()
    L = L_func(K)
    t2 = datetime.now()
    dt = (t2-t1).total_seconds()
    return dt

def test_lattice_time_by_lib(K, lib_name):
    if lib_name == 'concepts':
        L_time = test_lattice_time_by_func(K, lambda ctx: ctx.lattice)
    elif lib_name == 'fcapy':
        L_time = test_lattice_time_by_func(K, lambda ctx: ConceptLattice.from_context(ctx))
    #elif lib_name == 'fcapsy':
    #    L_time = test_lattice_time_by_func(K, lambda ctx: fcapsy.Lattice.from_context(ctx))
    else:
        raise ValueError(f'Given library "{lib_name}" is not supported')

    return L_time

def test_lattice_time_by_lib_multiprocess(K, lib_name, L_time):
    L_time.value = test_lattice_time_by_lib(K, lib_name)

In [20]:
import multiprocessing
def run_func_multiprocess(frame, lib_name, timeout_seconds):
    K = construct_context_by_lib(frame, lib_name)
    
    L_time, intent_time, extent_time = [multiprocessing.Value('f', -1, lock=False) for _ in range(3)]
    
    p = multiprocessing.Process(
        target=test_intent_extent_time_by_lib_multiprocess,
        name=f"test_intent_extent_{lib_name}",
        args=[frame, K, lib_name, intent_time, extent_time, 1000])
    p.start()
    p.join(timeout_seconds)
    if p.is_alive():    
        p.terminate()
        
    p = multiprocessing.Process(
        target=test_lattice_time_by_lib_multiprocess,
        name=f"test_lattice_{lib_name}",
        args=[K, lib_name, L_time])
    p.start()
    p.join(timeout_seconds)
    if p.is_alive():    
        p.terminate()
        
    def neg1_to_none(multiprocess_var):
        return multiprocess_var.value if multiprocess_var.value != -1 else None

    stat = {
        'lattice_construction_time (secs)': neg1_to_none(L_time),
        'intent_time (secs)': neg1_to_none(intent_time),
        'extent_time (secs)': neg1_to_none(extent_time),
        'timeout_seconds': timeout_seconds,
    }
    return stat

In [21]:
def get_context_stat(frame):
    K_stat = {
        'ctx_name': frame.name,
        'n_objects': frame.shape[0], 'n_attributes': frame.shape[1],
        'n_connections': frame.sum().sum(),
        'density': frame.sum().sum()/(frame.shape[0]*frame.shape[1]),
        'is_random': frame.name.startswith('random')
    }
    return K_stat

In [22]:
from tqdm.notebook import tqdm
import seaborn as sns

Run the benchmarks

In [23]:
frames_order = sorted(frames, key=lambda K_name: get_context_stat(frames[K_name])['n_connections'])

In [24]:
n_runs = 10
timeout_secs = 5*60

In [25]:
from itertools import product

In [26]:
run_number_vals = list(range(n_runs))
ctx_names_vals = frames_order
lib_names_vals = ['concepts', 'fcapy']#, 'fcapsy']
all_combs = list(product(run_number_vals, ctx_names_vals, lib_names_vals))
print(len(all_combs))

680


In [27]:
stats_df = pd.DataFrame(all_combs, columns=['run_number', 'ctx_name', 'lib_name'])
stats_df['is_computed'] = False
stats_df.to_csv('benchmark_stats_tmp.csv')

In [28]:
%%time

df_to_compute = stats_df[~stats_df['is_computed']][['run_number','ctx_name','lib_name']]

for comb in tqdm(df_to_compute.iterrows(), total=len(df_to_compute)):
    stats_df = pd.read_csv('benchmark_stats_tmp.csv', index_col=0)
    row_idx, (run_number, ctx_name, lib_name) = comb
    
    frame = frames[ctx_name]
    frame_stat = get_context_stat(frame)
    
    stat = run_func_multiprocess(frame, lib_name, timeout_seconds=timeout_secs)
    stat = dict(stat, **frame_stat)
    
    for k,v in stat.items():
        stats_df.loc[row_idx, k] = v
    stats_df.loc[row_idx, 'is_computed'] = True
    
    stats_df.to_csv('benchmark_stats_tmp.csv')

Process test_lattice_fcapy:
Traceback (most recent call last):
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-19-1f3dd588288a>", line 21, in test_lattice_time_by_lib_multiprocess
    L_time.value = test_lattice_time_by_lib(K, lib_name)
  File "<ipython-input-19-1f3dd588288a>", line 12, in test_lattice_time_by_lib
    L_time = test_lattice_time_by_func(K, lambda ctx: ConceptLattice.from_context(ctx))
  File "<ipython-input-19-1f3dd588288a>", line 3, in test_lattice_time_by_func
    L = L_func(K)
  File "<ipython-input-19-1f3dd588288a>", line 12, in <lambda>
    L_time = test_lattice_time_by_func(K, lambda ctx: ConceptLattice.from_context(ctx))
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/lattice/concept_lattice.py", line 230, in from_context
    ltc =

  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 154, in _sub_elements_cache
    res = self._sub_elements_nocache(element_index)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 147, in _sub_elements_nocache
    sub_indexes = {i for i in range(len(self)) if self.leq_elements(i, element_index) and i != element_index}
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 147, in <setcomp>
    sub_indexes = {i for i in range(len(self)) if self.leq_elements(i, element_index) and i != element_index}
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 263, in _leq_elements_cache
    self._cache_leq[key] = res
MemoryError
Process test_lattice_fcapy:
Traceback (most recent call last):
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.

  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/lattice.py", line 27, in __init__
    super(UpperSemiLattice, self).__init__(elements, leq_func, use_cache, direct_subelements_dict)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/lattice.py", line 105, in __init__
    bottom_elements = super(LowerSemiLattice, self).bottom_elements
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 122, in bottom_elements
    return [el_i for el_i in range(len(self)) if len(self.sub_elements(el_i)) == 0]
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 122, in <listcomp>
    return [el_i for el_i in range(len(self)) if len(self.sub_elements(el_i)) == 0]
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 154, in _sub_elements_cache
    res = self._sub_elements_nocache(element_index)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", 

  File "<ipython-input-19-1f3dd588288a>", line 12, in <lambda>
    L_time = test_lattice_time_by_func(K, lambda ctx: ConceptLattice.from_context(ctx))
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/lattice/concept_lattice.py", line 230, in from_context
    ltc = algo_func(context, **kwargs_used)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/algorithms/concept_construction.py", line 497, in lindig_algorithm
    lattice = ConceptLattice(concepts, subconcepts_dict=subconcepts_dict, superconcepts_dict=superconcepts_dict)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/lattice/concept_lattice.py", line 75, in __init__
    super(ConceptLattice, self).__init__(concepts, self.concepts_leq_func, use_cache=True)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/lattice.py", line 27, in __init__
    super(UpperSemiLattice, self).__init__(elements, leq_func, use_cache, direct_subelements_dict)
  File "/home/egor/anaconda3/lib/pytho

MemoryError
Process test_lattice_fcapy:
Traceback (most recent call last):
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-19-1f3dd588288a>", line 21, in test_lattice_time_by_lib_multiprocess
    L_time.value = test_lattice_time_by_lib(K, lib_name)
  File "<ipython-input-19-1f3dd588288a>", line 12, in test_lattice_time_by_lib
    L_time = test_lattice_time_by_func(K, lambda ctx: ConceptLattice.from_context(ctx))
  File "<ipython-input-19-1f3dd588288a>", line 3, in test_lattice_time_by_func
    L = L_func(K)
  File "<ipython-input-19-1f3dd588288a>", line 12, in <lambda>
    L_time = test_lattice_time_by_func(K, lambda ctx: ConceptLattice.from_context(ctx))
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/lattice/concept_lattice.py", line 230, in from_conte

  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 154, in _sub_elements_cache
    res = self._sub_elements_nocache(element_index)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 147, in _sub_elements_nocache
    sub_indexes = {i for i in range(len(self)) if self.leq_elements(i, element_index) and i != element_index}
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 147, in <setcomp>
    sub_indexes = {i for i in range(len(self)) if self.leq_elements(i, element_index) and i != element_index}
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 263, in _leq_elements_cache
    self._cache_leq[key] = res
MemoryError
Process test_lattice_fcapy:
Traceback (most recent call last):
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/egor/anaconda3/lib/python3.8/multiprocessing/process.

  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/lattice.py", line 27, in __init__
    super(UpperSemiLattice, self).__init__(elements, leq_func, use_cache, direct_subelements_dict)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/lattice.py", line 105, in __init__
    bottom_elements = super(LowerSemiLattice, self).bottom_elements
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 122, in bottom_elements
    return [el_i for el_i in range(len(self)) if len(self.sub_elements(el_i)) == 0]
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 122, in <listcomp>
    return [el_i for el_i in range(len(self)) if len(self.sub_elements(el_i)) == 0]
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", line 154, in _sub_elements_cache
    res = self._sub_elements_nocache(element_index)
  File "/home/egor/anaconda3/lib/python3.8/site-packages/fcapy/poset/poset.py", 


CPU times: user 15.4 s, sys: 7.71 s, total: 23.1 s
Wall time: 13h 8min 14s


In [29]:
!cp benchmark_stats_tmp.csv benchmark_stats.csv
!rm benchmark_stats_tmp.csv

# Analyze the results

In [30]:
stats_df = pd.read_csv('benchmark_stats.csv', index_col=0)
print(stats_df.shape)
stats_df.head()

(680, 13)


,run_number,ctx_name,lib_name,is_computed,lattice_construction_time (secs),intent_time (secs),extent_time (secs),timeout_seconds,n_objects,n_attributes,n_connections,density,is_random
0,0,random_10_10_0.1,concepts,True,0.000717,0.000005,0.000004,300.0,10.0,10.0,9.0,0.090,True
1,0,random_10_10_0.1,fcapy,True,0.002149,0.000004,0.000006,300.0,10.0,10.0,9.0,0.090,True
2,0,animal_movement,concepts,True,0.000831,0.000005,0.000004,300.0,16.0,4.0,24.0,0.375,False
3,0,animal_movement,fcapy,True,0.001956,0.000004,0.000007,300.0,16.0,4.0,24.0,0.375,False
4,0,gewaesser,concepts,True,0.001335,0.000005,0.000004,300.0,8.0,6.0,24.0,0.500,False


In [31]:
stats_df = stats_df.fillna(timeout_secs)

Benchmark results can be found in the file:
* _latice_construction_statistics.csv_

Contexts statistics (num. of objects, attributes, e.t.c) is in the file:
* _context_statistics.csv_

In [32]:
context_stat_feats = ['n_objects', 'n_attributes', 'n_connections', 'density']
context_stat_df = stats_df[['ctx_name',]+context_stat_feats].drop_duplicates().reset_index(drop=True)
context_stat_df.to_csv('context_statistics.csv')

We do not draw any plots in this notebook in order for Github to render it.

Benchmark plot can be found in the files:
* _lattice_construction_time_for_classic_contexts.png_
* _lattice_construction_time_for_random_contexts.png_
* _lattice_construction_time_all_data.png_

In [33]:
plt.rcParams['figure.facecolor'] = (1,1,1,1)  # (1,1,1,0)

In [34]:
y_feat = 'lattice_construction_time (secs)'
width = 2

plt.figure(figsize=(10,10))
for idx, x_feat in enumerate(context_stat_feats):
    plt.subplot(len(context_stat_feats)//width+1, width, idx+1)
    sns.lineplot(x=x_feat, y=y_feat, hue='lib_name', data=stats_df[~stats_df['is_random']])
    plt.xlabel(''); plt.ylabel('')
    plt.title(x_feat)
    plt.xlabel(x_feat)
    plt.ylabel('time (secs)')
    plt.axhline(timeout_secs, linestyle='--', color='grey') #label='maximal time per run')
    plt.text(plt.xlim()[1]*0.6, timeout_secs*1.05, 'maximal time per run')
    plt.ylim(-1, timeout_secs*1.2)
    plt.legend()
    
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.suptitle(f"{y_feat} based on context statistics\n(for classic fca contexts)")
plt.savefig('imgs/lattice_construction_time/classic_contexts.png', pad_inches=0.1, bbox_inches='tight')
plt.close()

In [35]:
context_stat_feats = ['n_objects', 'n_attributes', 'n_connections', 'density']
y_feat = 'lattice_construction_time (secs)'
width = 2

plt.figure(figsize=(10,10))
for idx, x_feat in enumerate(context_stat_feats):
    plt.subplot(len(context_stat_feats)//width+1, width, idx+1)
    sns.lineplot(x=x_feat, y=y_feat, hue='lib_name', data=stats_df[stats_df['is_random']])
    plt.xlabel(''); plt.ylabel('')
    plt.title(x_feat)
    plt.xlabel(x_feat)
    plt.ylabel('time (secs)')
    plt.axhline(timeout_secs, linestyle='--', color='grey') #label='maximal time per run')
    plt.text(plt.xlim()[1]*0.6, timeout_secs*1.05, 'maximal time per run')
    plt.ylim(-1, timeout_secs*1.2)
    plt.legend()
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.suptitle(f"{y_feat} based on context statistics\n(for random contexts)")
plt.savefig('imgs/lattice_construction_time/random_contexts.png', pad_inches=0.1, bbox_inches='tight')
plt.close()

In [36]:
stats_df['intent+extent_time (secs)'] = stats_df[['intent_time (secs)', 'extent_time (secs)']].sum(1)

In [37]:
context_stat_feats = ['n_objects', 'n_attributes', 'n_connections', 'density']
y_feat = 'intent+extent_time (secs)'
width = 2

plt.figure(figsize=(10,7))
for idx, x_feat in enumerate(context_stat_feats):
    plt.subplot(len(context_stat_feats)//width+1, width, idx+1)
    #sns.lineplot(x=x_feat, y=y_feat, hue='lib_name', data=stats_df)
    sns.lineplot(x=stats_df[x_feat], y=stats_df[y_feat]*1e6, hue=stats_df['lib_name'])
    plt.xlabel(''); plt.ylabel('')
    plt.title(x_feat)
    plt.xlabel(x_feat)
    plt.ylabel(r'time (microseconds)')
    plt.legend()
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.suptitle(f"{y_feat} based on context statistics\n(for random and classic contexts)")
plt.savefig('imgs/intent_extent_time/all_data.png', pad_inches=0.1, bbox_inches='tight')
plt.close()